In [63]:
import torch
from timeit import default_timer as timer

In [64]:
# If False pytorch has no access to your GPU
torch.cuda.is_available() and torch.cuda.device_count() > 0

True

In [65]:
target = "cuda"

In [66]:
initial_solution = torch.rand(20)

In [67]:
def calculate_cost(solution):
    # Implement your cost function here
    # Example: return the squared magnitude of the solution
    return torch.sum(solution ** 2)

In [68]:
def generate_neighbor(current_solution,target):
    # Implement neighbor generation based on your problem
    # Example: perturb the current solution by adding random noise
    perturbation = torch.randn_like(current_solution).to(target)
    neighbor = current_solution + perturbation
    return neighbor

In [69]:
def update_temperature(current_temperature):
    # Implement your cooling schedule here
    # Example: exponential decay
    cooling_rate = 0.95
    return current_temperature * cooling_rate

In [70]:
def calc_solution(initial_solution,target):
    current_solution = initial_solution.to(target)
    current_cost = calculate_cost(current_solution)
    best_solution = current_solution
    best_cost = current_cost
    current_temperature = 10

    for iteration in range(100):
        new_solution = generate_neighbor(current_solution,target)
        new_cost = calculate_cost(new_solution)
        delta_cost = new_cost - current_cost

        if delta_cost < 0 or torch.rand(1).to(target) < torch.exp(-delta_cost / current_temperature):
            current_solution = new_solution
            current_cost = new_cost

            if new_cost < best_cost:
                best_solution = new_solution
                best_cost = new_cost

        # Update temperature according to the annealing schedule
        current_temperature = update_temperature(current_temperature)

    # The best_solution contains the optimized solution
    del initial_solution, current_solution, new_solution
    return best_solution

In [71]:
%%timeit
best_solution = calc_solution(initial_solution, "cuda")

30.8 ms ± 2.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [73]:
%%timeit
best_solution = calc_solution(initial_solution, "cpu")

5.3 ms ± 50.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [75]:
best_solution = calc_solution(initial_solution, "cuda").to('cpu')

In [76]:
best_solution

tensor([0.5278, 0.7646, 0.3547, 0.5148, 0.6517, 0.6759, 0.0379, 0.1259, 0.2938,
        0.8681, 0.2359, 0.8354, 0.7280, 0.2047, 0.3082, 0.3361, 0.2581, 0.9345,
        0.0874, 0.2720])